In [1]:
#Installing and import needed packages
!pip install luwiji
import pandas as pd
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from nltk.tokenize import word_tokenize
from luwiji.recommendation_system import illustration

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 400.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
  Created wheel for luwiji: filename=luwiji-1.2.1-py2.py3-none-any.whl size=40366830 sha256=357ba1a24cec0825a5d366c54fb6fa34ba3b77e71bafc68dc9bc037931327c4b
  Stored in directory: /root/.cache/pip/wheels/ba/90/9e/943ac5067e9cdc185d8f029a1d13278acd52839118acc06c9a
  Created wheel for jcopml: filename=jcopml-1.2.2-py2.py3-none-any.whl size=32178 sha256=2e55cce4dfd96e47f719ba8af6560bccda644d2162431dd3911a55dba47985da
  Stored in directory: /root/.cache/pip/wheels/5e/06/58/749e5f1baf2932285758798a92793d4347888826160b025808
Successfully built luwiji jcopml


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
#Upload the menu datasets to collab
from google.colab import files
uploaded = files.upload()

Saving Book2.csv to Book2.csv


In [3]:
#Read the datasets into variables menu_data
menu_data=pd.read_csv("./Book2.csv",delimiter=';')
menu_data

,Recipe_name,Diet_type,Cuisine_type,Protein(g),Carbs(g),Fat(g)
0,Bone Broth From 'Nom Nom Paleo',paleo,american,5.22,1.29,3.20
1,"Paleo Effect Asian-Glazed Pork Sides, A Sweet ...",paleo,south east asian,181.55,28.62,146.14
2,Paleo Pumpkin Pie,paleo,american,30.91,302.59,96.76
3,Strawberry Guacamole recipes,paleo,mexican,9.62,75.78,59.89
4,"Asian Cauliflower Fried ""Rice"" From 'Nom Nom P...",paleo,chinese,39.84,54.08,71.55
...,...,...,...,...,...,...
7799,Brown Butter-Sunchoke Soup With Brussels Sprou...,dash,italian,85.20,288.14,137.15
7800,Make-Your-Own-Salad,dash,american,141.98,123.18,323.50
7801,Luis Buñuel Dry Martini,dash,world,0.01,0.39,0.00
7802,Cornflake Semi-Fried Chicken Tenders,dash,american,155.38,239.88,260.84


In [4]:
#Deleting the tittle of menu datasets and save it to menu_df for creating metadata
#Creating metadata column as references for recommendation
menu_df=pd.DataFrame(menu_data.drop(columns=["Recipe_name"]))
menu_df['metadata'] = ''
for i in menu_df.columns[:-1]:
  menu_df['metadata'] += ' ' + menu_df[i].map(str)

In [5]:
#Creating RecommenderSystem class for creating recommeder system based on word tokenizer
#Create init function to define the datasets we use and content column as a references
#Create fit function to train datasets with creating token for content column using word tokenizer
#Create recommend function to give recommendation based on cosine distances from the input and metadata (content column) in datasets

class RecommenderSystem:
  def __init__(self, data, content_col):
    self.df=data
    self.content_col=content_col
    self.encoder = None
    self.bank = None

  def fit (self):
    self.encoder= CountVectorizer(stop_words='english', tokenizer=word_tokenize)
    self.bank = self.encoder.fit_transform(self.df[self.content_col])

  def recommend(self, metadata_input, topk=5):
    content=metadata_input
    code =self.encoder.transform ([content])
    dist =cosine_distances(code, self.bank)
    rec_idx=dist.argsort()[0, 1:(topk+1)]
    return menu_data.loc[rec_idx]

In [6]:
#Initiating recsys as RecommenderSystem class
#Training datasets with recsys.fit()
recsys=RecommenderSystem(data=menu_df, content_col='metadata')
recsys.fit()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [7]:
#Creating an example input from the datasets itself
input_1=menu_df.iloc[7802,-1]
input_2=menu_df.iloc[455,-1]
menu_df

,Diet_type,Cuisine_type,Protein(g),Carbs(g),Fat(g),metadata
0,paleo,american,5.22,1.29,3.20,paleo american 5.22 1.29 3.2
1,paleo,south east asian,181.55,28.62,146.14,paleo south east asian 181.55 28.62 146.14
2,paleo,american,30.91,302.59,96.76,paleo american 30.91 302.59 96.76
3,paleo,mexican,9.62,75.78,59.89,paleo mexican 9.62 75.78 59.89
4,paleo,chinese,39.84,54.08,71.55,paleo chinese 39.84 54.08 71.55
...,...,...,...,...,...,...
7799,dash,italian,85.20,288.14,137.15,dash italian 85.2 288.14 137.15
7800,dash,american,141.98,123.18,323.50,dash american 141.98 123.18 323.5
7801,dash,world,0.01,0.39,0.00,dash world 0.01 0.39 0.0
7802,dash,american,155.38,239.88,260.84,dash american 155.38 239.88 260.84


In [8]:
input_1

' dash american 155.38 239.88 260.84'

In [9]:
recsys.recommend(input_1)

,Recipe_name,Diet_type,Cuisine_type,Protein(g),Carbs(g),Fat(g)
7787,Cornflake Semi-Fried Chicken Tenders,dash,american,155.38,239.88,260.84
7803,Emeril's Classic Manhattan,dash,american,0.02,0.83,0.00
6445,Wine-Poached Peaches,dash,american,11.54,161.96,59.71
7168,The last of the good pears tart,dash,american,16.87,255.14,93.35
6448,Pumpkin Soup,dash,american,56.71,299.68,69.76


In [10]:
input_2

' paleo american 35.46 39.47 97.69'

In [11]:
recsys.recommend(input_2)

,Recipe_name,Diet_type,Cuisine_type,Protein(g),Carbs(g),Fat(g)
0,Bone Broth From 'Nom Nom Paleo',paleo,american,5.22,1.29,3.20
629,Waldorf Salad recipes,paleo,american,46.42,75.17,187.33
631,Garlic Mashed Sweet Potatoes With Spinach recipes,paleo,american,25.05,200.19,30.70
634,Low Carb Cauliflower Potato Salad recipes,paleo,american,26.18,74.50,62.80
635,Paleo Chili,paleo,american,305.91,139.01,356.15


In [17]:
from joblib import dump

dump(recsys.encoder, 'encoder.joblib')
dump(recsys.bank, 'bank.joblib')
dump(menu_data, 'menu_data.joblib')

['menu_data.joblib']